In [ ]:
import pandas as pd
import json
import requests
from datetime import datetime

# data fetching...............

offset_val = 0

# Define the base URL without filter parameters
base_url = "https://data.exactspace.co/exactapi/units/60ae9143e284d016d3559dfb/activities"

all_objects = []  # List to store all fetched objects

while len(all_objects) < 900:  # Continue fetching until desired number is reached
    
    # Construct the URL with the updated offset value
    url = f"{base_url}?filter=%7B%20%20%20%20%20%22where%22%3A%20%7B%20%20%20%20%20%20%20%20%20%22type%22%3A%20%22task%22%20%20%7D%2C%20%20%20%20%20%22order%22%3A%20%22createdOn%20DESC%22%2C%20%20%20%20%20%22limit%22%3A%2010%2C%20%22offset%22%3A{offset_val}%20%7D&access_token=Ziv35SH8DYvWbh0ZL8HMhOCESfQTwajS7j8iHfb1A3pEoSO6P8NraCLmCD1g7iAH"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        objects_fetched = len(data)  # Number of objects fetched in this request
        all_objects.extend(data)      # Add fetched objects to the list
        offset_val += 10
        print("offset success:", offset_val)
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        break

print("Total objects fetched:", len(all_objects))


# recommendation sheet........

tagList = ["GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric"]

def getValues4(tagList, startDate, endDate):
    url =  "https://data.exactspace.co/kairosapi/api/v1/datapoints/query"
    d = {
        "metrics": [
            {
                "tags": {},
                "name": "",
            }
        ],
        "plugins": [],
        "cache_time": 0,
        "start_absolute": startDate,
        "end_absolute": endDate
    }
    dfs = []
    for tag in tagList:
        d['metrics'][0]['name'] = tag
        res = requests.post(url=url, json=d)
        values = json.loads(res.content)
        df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
        df['time'] = pd.to_datetime(df['time'], unit='ms') + pd.Timedelta(hours=5.5)
        df.sort_values(by='time', inplace=True)
        df = df.drop_duplicates(subset=["time", tag])
        if df.shape[0] < 10:
            pass
        else:
            dfs.append(df)
    df = dfs[0]
    for df_ in dfs[1:]:
        df = pd.merge(df, df_, on='time')
    return df

# Function to convert ISO 8601 timestamp to epoch timestamp
def iso_to_epoch(iso_timestamp):
    return int(datetime.fromisoformat(iso_timestamp[:-1]).timestamp())

# Extract recommendation data from JSON and create DataFrame
def extract_recommendations(all_objects):
    recommendations = []
    for obj in all_objects:
        title = obj.get("content", [{}])[0].get("value", "")
        if "GAP: GSD Recommendations" in title:
            recommendation_text = obj.get("content", [{}])[1].get("value", "")  # Assuming the recommendation text is in the second item of the content list
            update_history = obj.get("updateHistory", [])
            update_actions = [entry.get("action", "") for entry in update_history]  # Extract actions from updateHistory
            actions_str = "\n".join(update_actions)  # Concatenate actions into a single string separated by newline
            created_time = obj.get("createdOn")
            due_time = obj.get("dueDate")
            startDate = iso_to_epoch(created_time) * 1000
            endDate = iso_to_epoch(due_time) * 1000
            # Fetch data for the specified time range
            data = getValues4(tagList, startDate, endDate)
            data = data.tail(1)
            data = data.round(4)
            data = data.drop(columns=['time'])
            density = data.to_dict('records')
            result = density[0].get("GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric")
            if result > 1.65:
                density_achieved = "Yes"
            else:
                density_achieved = "No"
            recommendations.append({"ID": obj.get("id", ""), "Recommendation": recommendation_text, "Density Achieved": density_achieved, "Update Actions": actions_str})
    return recommendations

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    recommendations = extract_recommendations(all_objects)
    if not recommendations:
        print("No recommendations found.")
        return
    
    df = pd.DataFrame(recommendations)
    
    # Define writer
    writer = pd.ExcelWriter("recommendations.xlsx", engine="xlsxwriter")
    df.to_excel(writer, index=False, sheet_name="Recommendations")
    worksheet = writer.sheets["Recommendations"]
    
    # Set column width
    worksheet.set_column('A:Z', 15)  # Set column width for ID
    
    # Increase row height for cells with long text
    for i, text in enumerate(df["Recommendation"], start=1):
        num_lines = text.count("\n") + 1  # Count the number of lines in the text
        if num_lines > 1:
            worksheet.set_row(i, 20 * num_lines)  # Set row height to accommodate the text
        
    writer.save()
    print("Excel file saved successfully.")

# Assuming `all_objects` is defined somewhere
extract_and_write_to_excel(all_objects)

# Load the Excel file
excel_file = 'recommendations.xlsx'
df = pd.read_excel(excel_file)

# Define the phrase to remove
phrase_to_remove = "This task is created by Pulse."

# Remove the phrase from the specified column
df['Update Actions'] = df['Update Actions'].str.replace(phrase_to_remove, '')

# Save the modified DataFrame back to Excel
output_file = 'final_report.xlsx'
df.to_excel(output_file, index=False)

print("Data cleaned and saved successfully.")


In [2]:
import pandas as pd
import json
import requests
from datetime import datetime

def getValues(tagList, start ,end):
        url = "https://data.exactspace.co/kairosapi/api/v1/datapoints/query"
        d = {
            "metrics": [
                {
                    "tags": {},
                    "name": ""
                }
            ],
            "plugins": [],
            "cache_time": 0,
            "cache_time": 0,
            "start_absolute": start,
            "end_absolute": end
        }
        finalDF = pd.DataFrame()
        for tag in tagList:
            d['metrics'][0]['name'] = tag
            res = requests.post(url=url, json=d)
            values = json.loads(res.content)
            df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
            finalDF = pd.concat([finalDF, df], axis=1)

        finalDF = finalDF.loc[:, ~finalDF.columns.duplicated()]
        finalDF.dropna(subset=['time'], inplace=True)
        # finalDF['time'] = pd.to_datetime(finalDF['time'], unit='ms').dt.strftime('%d-%m-%y %H:%M:%S')
        return finalDF

def mld_2_anode_status(start, end):
    tags= [
            'GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric',
            'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
            'GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'
        ]
    
    temp_df = pd.DataFrame(columns=['time'])
    temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
    temp_df.loc[1, 'time'] = end
    temp_df['time'] += (5 * 60 + 30) * 60 * 1000
    temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

    mld_2_df=getValues(tags, start, end)
    
    mld_2_df = mld_2_df.dropna()
    mld_2_df = mld_2_df[(mld_2_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_2_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.56) & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] <= 1.69)]
    
    mld_2_df = mld_2_df[mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] % 1 == 0]
    mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].astype(int)

    # Find consecutive duplicate values in the column and mark them for removal
    mld_2_df['to_remove'] = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] == mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].shift(1)

    mld_2_df = mld_2_df[mld_2_df['to_remove'] != True]
    
    mld_2_df.reset_index(drop=True, inplace=True)
    # print(mld_2_df)
    # Assuming df is your DataFrame
    # Convert the 'time' column to datetime format
    mld_2_df['time'] += (5 * 60 + 30) * 60 * 1000
    mld_2_df['date'] = pd.to_datetime(mld_2_df['time'], unit='ms')

    # Define conditions for categorizing 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'
    condition_better = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.66
    condition_good = (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.65) & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] < 1.66)
    condition_bad = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] < 1.65

    # Create a new column 'category' based on conditions
    mld_2_df['category'] = 'Undefined'
    mld_2_df.loc[condition_better, 'category'] = 'Better'
    mld_2_df.loc[condition_good, 'category'] = 'Good'
    mld_2_df.loc[condition_bad, 'category'] = 'Bad'

    # Create an empty DataFrame with columns
    hourly_counts = pd.DataFrame(columns=['Bad', 'Better', 'Good', 'time'])

    # Group by hour and category, then count occurrences
    hourly_counts = mld_2_df.groupby([mld_2_df['date'].dt.floor('H'), 'category']).size().unstack(fill_value=0)

    complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
    reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

    # Add columns for 'Good', 'Better', and 'Bad' with zeros
    reference_df['Good'] = 0
    reference_df['Better'] = 0
    reference_df['Bad'] = 0

    missing_categories = set(['Bad', 'Better', 'Good']) - set(hourly_counts.columns)
    for category in missing_categories:
            hourly_counts[category] = 0

    hourly_counts = pd.merge(reference_df, hourly_counts, how='left', on=['date'])
    hourly_counts = hourly_counts[['date', 'Good_y', 'Better_y', 'Bad_y']]
    hourly_counts = hourly_counts.rename(columns={'Good_y': 'Good', 'Better_y': 'Better', 'Bad_y':'Bad'})

    hourly_counts = hourly_counts.fillna(0)
    return hourly_counts

def mld_1_anode_status(start, end):
        tags= [
                'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
                'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
                'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'
            ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
            & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)

        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')

        # Define conditions for categorizing 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'
        condition_better = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.66
        condition_good = (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.65) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] < 1.66)
        condition_bad = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] < 1.65

        # Create a new column 'category' based on conditions
        mld_1_df['category'] = 'Undefined'
        mld_1_df.loc[condition_better, 'category'] = 'Better'
        mld_1_df.loc[condition_good, 'category'] = 'Good'
        mld_1_df.loc[condition_bad, 'category'] = 'Bad'

        # Create an empty DataFrame with columns
        hourly_counts = pd.DataFrame(columns=['Bad', 'Better', 'Good', 'time'])

        # Group by hour and category, then count occurrences
        hourly_counts = mld_1_df.groupby([mld_1_df['date'].dt.floor('H'), 'category']).size().unstack(fill_value=0)

        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        # Add columns for 'Good', 'Better', and 'Bad' with zeros
        reference_df['Good'] = 0
        reference_df['Better'] = 0
        reference_df['Bad'] = 0

        missing_categories = set(['Bad', 'Better', 'Good']) - set(hourly_counts.columns)
        for category in missing_categories:
                hourly_counts[category] = 0

        hourly_counts = pd.merge(reference_df, hourly_counts, how='left', on=['date'])
        hourly_counts = hourly_counts[['date', 'Good_y', 'Better_y', 'Bad_y']]
        hourly_counts = hourly_counts.rename(columns={'Good_y': 'Good', 'Better_y': 'Better', 'Bad_y':'Bad'})

        hourly_counts = hourly_counts.fillna(0)
        return hourly_counts

def kpis_dashboard(start, end):
        tags= [
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
        'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density'
        ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)

        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')
        mld_1_df.set_index('date', inplace=True)


        columns_of_interest = [
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height'
        ]

        # Calculate hourly mean for multiple columns
        hourly_mean = mld_1_df[columns_of_interest].resample('H').mean()
        hourly_std = mld_1_df[columns_of_interest].resample('H').std()
        hourly_std = hourly_std.round(3)
        hourly_mean = hourly_mean.round(3)

        hourly_std.reset_index(inplace=True)
        hourly_mean.reset_index(inplace=True)


        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        hourly_mean = pd.merge(reference_df, hourly_mean, how='left', on=['date'])
        hourly_std = pd.merge(reference_df, hourly_std, how='left', on=['date'])
        global test_df
        test_df = hourly_std.copy()

        missing_categories1 = set(['GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']) - set(hourly_mean.columns)

        missing_categories2 = set(['GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']) - set(hourly_std.columns)

        for category in missing_categories1:
                hourly_mean[category] = 0

        for category in missing_categories2:
                hourly_std[category] = 0

        hourly_mean = hourly_mean.fillna(0)
        hourly_std = hourly_std.fillna(0)

        hourly_mean.set_index('date', inplace=True)
        hourly_std.set_index('date', inplace=True)

        # Convert the index to epoch format
        hourly_mean["time"] = hourly_mean.index.astype(int) // 10**6  
        hourly_mean["time"] -= (5 * 60 + 30) * 60 * 1000

        hourly_std["time"] = hourly_std.index.astype(int) // 10**6  
        hourly_std["time"] -= (5 * 60 + 30) * 60 * 1000

        return hourly_mean, hourly_std

start = 1701369000000
end = 1703961000000

hourly_mld2_anode_status = mld_2_anode_status(start, end)
mld2_bad = hourly_mld2_anode_status['Bad'].sum()
mld2_better = hourly_mld2_anode_status['Better'].sum()
mld2_good = hourly_mld2_anode_status['Good'].sum()

hourly_mld1_anode_status = mld_1_anode_status(start, end)
mld1_bad = hourly_mld1_anode_status['Bad'].sum()
mld1_better = hourly_mld1_anode_status['Better'].sum()
mld1_good = hourly_mld1_anode_status['Good'].sum()

hourly_means_kpis, hourly_std_kpis = kpis_dashboard(start, end)

# # Define shorter column names
# new_column_names = {
#     'GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight': 'Weight',
#     'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density': 'Dry_Density',
#     'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric': 'Geometric',
#     'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height': 'Height'
# }

# columns_to_format = ['GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight', 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density', 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric', 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']

# # Format the selected columns
# formatted_series = hourly_std_kpis[columns_to_format].applymap(lambda x: "{:.3f}".format(x))

# # Rename the columns
# formatted_and_renamed_series_kpis = formatted_series.rename(columns=new_column_names)

In [36]:
hourly_means_kpis, hourly_std_kpis = kpis_dashboard(start, end)

c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:259: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:262: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


In [37]:
# Define shorter column names
new_column_names = {
    'GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight': 'Anode Weight',
    'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density': 'Anode Dry Density',
    'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric': 'Anode Geometric Density',
    'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height': 'Anode Height'
}

In [40]:
hourly_means_kpis = hourly_means_kpis.rename(columns=new_column_names)

In [45]:
hourly_means_kpis[hourly_means_kpis['Anode Dry Density']>0].round(4).mean()

Anode Weight               1.065279e+03
Anode Dry Density          1.439107e+00
Anode Geometric Density    1.654734e+00
Anode Height               6.804077e+02
time                       1.702562e+12
dtype: float64